In [1]:
import scipy.io
import pandas as pd
import numpy as np
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

path_to_annot_file = "/home/eonrrfe/Documents/Repos/Others/CarDetection/test_images/Standford standard car set/archive/cars_annos.mat"

new_mat = scipy.io.loadmat(path_to_annot_file)
trial = np.hstack(new_mat['annotations'])
trial_df = pd.DataFrame(trial)
trial_df['file_name'] = trial_df['relative_im_path']
trial_df['file_name'] = trial_df['file_name'].map(np.array2string)
trial_df['file_name'] = trial_df['file_name'].map(lambda x:x.lstrip("['car_ims/").rstrip("']"))
trial_df['class_number'] = trial_df['class'].astype(float)
names= new_mat['class_names'].copy()
names_df = pd.DataFrame(names).T
names_df['class_number'] =names_df.index+1
names_df['car_name'] = names_df[0].map(lambda x:np.array2string(x).lstrip("['").rstrip("']"))
names_df = names_df.drop(columns = 0)
names_df['brand'] = names_df['car_name'].map(lambda x:x.split()[0])
names_df['model'] = names_df['car_name'].map(lambda x:x.split()[1])
names_df['both']= names_df['brand'] +  names_df['model']

2024-08-23 06:26:48.047831: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-23 06:26:48.049106: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-23 06:26:48.051930: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-23 06:26:48.058835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 06:26:48.070574: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
names_df

,class_number,car_name,brand,model,both
0,1,AM General Hummer SUV 2000,AM,General,AMGeneral
1,2,Acura RL Sedan 2012,Acura,RL,AcuraRL
2,3,Acura TL Sedan 2012,Acura,TL,AcuraTL
3,4,Acura TL Type-S 2008,Acura,TL,AcuraTL
4,5,Acura TSX Sedan 2012,Acura,TSX,AcuraTSX
...,...,...,...,...,...
191,192,Volkswagen Beetle Hatchback 2012,Volkswagen,Beetle,VolkswagenBeetle
192,193,Volvo C30 Hatchback 2012,Volvo,C30,VolvoC30
193,194,Volvo 240 Sedan 1993,Volvo,240,Volvo240
194,195,Volvo XC90 SUV 2007,Volvo,XC90,VolvoXC90


In [4]:

names_df['both'] = names_df['both'].replace('RamC/V','RamCV')#the '/' causes issues when creating folders so it is deleted

names_df['both'] = names_df['both'].map(lambda x : x.lower())

carBrand = names_df['both'].unique()

for i in carBrand:
  os.makedirs(
    "/home/eonrrfe/Documents/Repos/Others/CarDetection/test_images/Standford standard car set/apm_cnn_trial/trial_2/test_brand_and_model_2/{}".format(i)
  )


# CONTINUE:
# - try to use the 50-50 set to train the model
# - if cannot then try to get back the original full dataset

# links: https://github.com/ijosephp/apm_project/blob/main/Resnet50.ipynb
#       https://patricia-schutter.medium.com/car-image-recognition-with-convolutional-neural-network-applications-e791c98c9d72
